# Software Development und Python Basics

In diesem Notebook sammle ich ein paar Basics zum Entwickeln eines Projektes in Python.
Da ihr schon Kurse mit Python hattet, sollte das meiste nicht neu sein, eher sollen wichtige Parts wiederholt werden.
Wenn für euch noch wichtige Themen fehlen, kann ich diese hier noch hinzufügen. 

In [1]:
import datetime
import random
from pathlib import Path

import pandas as pd

## Organisierung eines Python Projekts

Eine kurze Übersicht, wie man seinen Code organisieren kann, von ersten Schritten und einfachen Programmen zu Komponenten die gut wiederverwendet werden können.

### Notebook

Als erste Methode, um schnell Sachen auszuprobieren und Ergebnisse sofort anzuzeigen, eignen sich Notebooks sehr gut.
Wenn immer wieder aufgeräumt wird und auch die anderen Features wie Markdown Überschriften und Text verwendet werden,
eignen sich diese auch sehr gut zum Anschaulich machen der einzelnen Schritte und um neues zu testen.
Da man aber oft an unterschiedlichen Stellen weiterarbeitet oder etwas neues probiert, können diese auch recht
schnell sehr unübersichtlich werden.

In [2]:
# Tipp um eigene Funktionen automatisch neu zu laden
%load_ext autoreload
%autoreload 2

### Skript

Wenn der Code, wie im Notebook definiert, in ein Python File kopiert wird, so dass dieser von oben nach unten
ausgeführt wird, hat man ein Skript. In den meisten Fällen werdet ihr die Hauptschritte eures Projektes
in einem Skript ausführen. Idealerweise werden die wichtigen Komponenten hier aber nur aufgerufen, 
und befinden sich schön gegliedert in anderen Files.

### Funktionen

Sobald die gewünschte Herangehensweise im Notebook ausgetestet wurde, macht es Sinn diese in 
einzelne Funktionen aufzuteilen. Diese haben jeweils eine klar beschrieben Aufgabe, definierte 
Input und Output Parameter und können somit auch wiederverwendet werden. 

Meine Empfehlung ist es, sich bevor man den Code in die Funktion gibt, zu überlegen welche Ein- und
Ausgangsparameter man erwartet. Am besten geht das, in dem man zuerst die Signatur der Funktion
inklusive Docstring schreibt, und dann die Funktionalität aus dem Notebook überträgt, oft mit
einigen Verallgemeinerungen.

In [3]:
# Example of a Function stub, to define functionality, in and output parameters before the implementation
def load_profile_for_day(  # Descriptive Name
    day: datetime.date,
    profile_df: pd.DataFrame,
    dynamize: bool = False,  # Parameters, with type hints to make explicit what we expect
) -> pd.Series:  # Return type
    # Docstring with one title line, optionally some more description, and a description of Arguments and what is returned
    """Get the 15 minute load profile for a single day.

    Args:
        day (datetime.date): Date for which to get the load profile.
        profile_df (pd.DataFrame): In the format as given by get_profile_from_file.
        dynamize (bool): Should dynamization by the 4th-grade polynomial dynamization
            function for day of year be applied. This should be only done for H0. Defaults to False.

    Returns:
        pd.Series: Series with Watt values every 15 minutes, and a pd.DatetimeIndex.
    """
    pass  # Implementation follows in second step.

Diese Herangehensweise habe ich für die Standardlastprofile gewählt. Im Notebook *11_load_profiles.ipynb*
habe ich die einzelnen Schritte ausprobiert, dann habe ich eine Funktion erstellt und diese 
schlussendlich in ein eigenes File *src/load_profiles.py* kopiert. Von dort können sie nun importiert werden.

In [4]:
from src.load_profiles import get_load_profile

In [5]:
lastprofile_file = (
    Path(".") / "external" / "Lastprofile" / "representative_profiles_vdew.xls"
)
assert (
    lastprofile_file.is_file()
), f"Did not find file with representative load profiles at {lastprofile_file}"

In [6]:
h0profile = get_load_profile(
    lastprofile_file,
    from_=datetime.date(2015, 1, 1),
    to=datetime.date(2015, 12, 31),
    type="H0",
)

In [7]:
h0profile.head()

2015-01-01 00:15:00    108.677635
2015-01-01 00:30:00    100.728643
2015-01-01 00:45:00     93.152259
2015-01-01 01:00:00     85.824281
2015-01-01 01:15:00     78.744710
dtype: float64

### Classes

Objekt-orientiertes Programmieren baut sehr stark auf Klassen auf. Vor allem nützlich wenn Objekte
erstellt werden sollen, die auch ihren eigenen Zustand speichern sollen.
Klasse am besten in einem eigenen File erstellen, hier ein kurzes Beispiel als Wiederholung.

In [8]:
# Class Names by convention in CamelCase
# Inheriting from default class object
class Wuerfel(object):
    # properties can be defined statically here
    seiten = [1, 2, 3, 4, 5, 6]
    
    # __init__ method is called when the class is instantiated
    # Usually used to define properties dynamically
    # Every method has self as it's first argument, this is used to access all class properties and methods
    def __init__(self, init_seite_oben: int=None):
        if init_seite_oben is None:
            self.seite_oben = random.choice(self.seiten)
        else:
            self.seite_oben = init_seite_oben
    
    # methods, self and possibly more arguments, optional return
    def wurf(self):
        self.seite_oben = random.choice(self.seiten)

    def setzen(self, seite_oben: int):
        self.seite_oben = seite_oben
        
    def check_seite_oben(self) -> bool:
        if self.seite_oben in self.seiten:
            return True
        else:
            return False

In [9]:
# Instantiate 1 die
d1 = Wuerfel(1)
d1.seite_oben

1

In [10]:
d1.wurf()
d1.seite_oben

6

In [11]:
# Can create multiple instances of dice
d2 = Wuerfel()
d2.seite_oben

2

In [12]:
# d1 is independent
d1.seite_oben

6

In [13]:
print(d2.check_seite_oben())
d2.setzen(7)
print(d2.check_seite_oben())

True
False


In [14]:
# Inspect class instance
dir(d2)
# __name__ methods are thunder methods that are defined by object. They are used for various operations
# Sometimes you want to redefine them, so you get special behaviour.
# _name methods and properties are usually private, so not intended to be used
# all our defined properties and methods are in the end

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'check_seite_oben',
 'seite_oben',
 'seiten',
 'setzen',
 'wurf']

In [15]:
# Overwrite the str method, that defines what happens when the class is printed / converted to string
print(d2)

In [16]:
def wuerfel_str(self):
    return str(self.seite_oben)

# Adding methods outside of the class definition is not good practice
Wuerfel.__str__ = wuerfel_str

In [17]:
print(d2)

7


## Timeseries

Wir arbeiten mit Zeitserien, Diskretisierung in Intervalle, zB 15 Minuten Werte.

Python Tool: pandas DataFrame mit DateTimeIndex. 

### Grafische Libraries zum Arbeiten mit Pandas

* bamboolib
* pandasgui
* mito

In [18]:
dti = pd.date_range(
    "2020-01-01 00:00:00", "2021-01-01 00:00:00", tz="utc", freq="15min"
)
dti

DatetimeIndex(['2020-01-01 00:00:00+00:00', '2020-01-01 00:15:00+00:00',
               '2020-01-01 00:30:00+00:00', '2020-01-01 00:45:00+00:00',
               '2020-01-01 01:00:00+00:00', '2020-01-01 01:15:00+00:00',
               '2020-01-01 01:30:00+00:00', '2020-01-01 01:45:00+00:00',
               '2020-01-01 02:00:00+00:00', '2020-01-01 02:15:00+00:00',
               ...
               '2020-12-31 21:45:00+00:00', '2020-12-31 22:00:00+00:00',
               '2020-12-31 22:15:00+00:00', '2020-12-31 22:30:00+00:00',
               '2020-12-31 22:45:00+00:00', '2020-12-31 23:00:00+00:00',
               '2020-12-31 23:15:00+00:00', '2020-12-31 23:30:00+00:00',
               '2020-12-31 23:45:00+00:00', '2021-01-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=35137, freq='15T')

In [19]:
# df, pvlib, whatever Daten

In [20]:
# Timezones, resampling, missing values

## Visualisierung

Möglichkeiten:

* matplotlib
* seaborn
* plotly

## Daten

* [APCS](https://www.apcs.at/de/clearing/technisches-clearing/lastprofile)
* [OPSD](https://open-power-system-data.org/)
* https://solar.htw-berlin.de/elektrische-lastprofile-fuer-wohngebaeude/
* https://www.renewables.ninja/
* https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis_en
* https://simbench.de/de/datensaetze/

## Libraries

### Anbindung Wiener Netze Smart Meter Daten

Wiener Netze Smart Meter Portal https://smartmeter-web.wienernetze.at/#/welcome.

Git Repository fuer Abruf ueber API: https://github.com/platysma/vienna-smartmeter

## Best Practices

* Docstrings
* Type Hints
* Start with Notebook, move important pieces to .py file
* black autoformatter
* Exception Handling
* autoreload